In [1]:
import pandas as pd; import numpy as np; import plotly.express as px; import wget

In [2]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
spacex_data = wget.download(url)
data = pd.read_csv(spacex_data)

100% [............................................................] 2476 / 2476

In [3]:
data.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Flight Number             56 non-null     int64  
 1   Launch Site               56 non-null     object 
 2   class                     56 non-null     int64  
 3   Payload Mass (kg)         56 non-null     float64
 4   Booster Version           56 non-null     object 
 5   Booster Version Category  56 non-null     object 
dtypes: float64(1), int64(2), object(3)
memory usage: 2.8+ KB


In [5]:
data.columns

Index(['Flight Number', 'Launch Site', 'class', 'Payload Mass (kg)',
       'Booster Version', 'Booster Version Category'],
      dtype='object')

In [6]:
data.isnull().sum()

Flight Number               0
Launch Site                 0
class                       0
Payload Mass (kg)           0
Booster Version             0
Booster Version Category    0
dtype: int64

In [7]:
data['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [8]:
data.head(2)

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0


In [9]:
data['Payload Mass (kg)'].max()

9600.0

In [10]:
from jupyter_dash import JupyterDash
from dash import dcc,html,Input, Output,State


In [11]:
app = JupyterDash(__name__)
app.layout = html.Div(children=[html.H1('Spacex Launch Records Dashboard',
                                        style={'textAlign':'center','color':'#503D36','font-size':24}),
                               html.Div([dcc.Dropdown(id='site',
                                                      options=[{'label':'All Sites','value':'All'},
                                                               {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                                               {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                                               {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                                               {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'}],
                                                      placeholder='Select a launch Site',
                                                      searchable=True
                                                     ),
                                         
                                        ]),
                                html.Div([],id='pie_chart'),
                                html.Div([html.H5('Payload Mass (Kg)')]),
                                
                                html.Div([dcc.RangeSlider(id='Payload_slider',
                                                         min=0,
                                                         max=10000,
                                                         step=1000,
                                                         value=[0,9600]),
                                                          
                                         ]),
                                html.Div([],id='scatter_chart'),
                                
                               ])

@app.callback([Output(component_id='pie_chart',component_property='children'),
              Output(component_id='scatter_chart',component_property='children')],
              [Input(component_id='site',component_property='value'),
              Input(component_id='Payload_slider',component_property='value')],
              [State('pie_chart','children'),State('scatter_chart','children')]
              )
def get_graph(enter_site,payload,c1,c2):
    if enter_site =='All':
        spacex_df =data
        fig_all = px.pie(spacex_df,values='class',names='Launch Site',
                         title='Total Success Launches for All Sites')
        spacex_load =data[(data['Payload Mass (kg)'] >= payload[0]) & 
                          (data['Payload Mass (kg)'] <= payload[1])] 
        fig_scatter = px.scatter(spacex_load, x='Payload Mass (kg)',
                                 y='class',color='Booster Version Category',
                                title='Correlation between Payload and Success for all Sites')
        return [dcc.Graph(figure=fig_all),dcc.Graph(figure=fig_scatter)]
    else:
        spacex_df = data[data['Launch Site']== enter_site]
        fig = px.pie(spacex_df,values='class',names='Launch Site',
                     title="Total Success Launches for {}".format(enter_site))
        spacex_load =data[(data['Payload Mass (kg)'] >= payload[0]) &
                          (data['Payload Mass (kg)'] <= payload[1])] 
        fig_scatter = px.scatter(spacex_load,x='Payload Mass (kg)',
                                 y='class',color='Booster Version Category',
                                title="Correlation between Payload and Success for{}".format(enter_site))
        return [dcc.Graph(figure=fig),dcc.Graph(figure=fig_scatter)]
if __name__ == '__main__':
    app.run_server(port=8079,mode='inline',debug=True)
    
    


Dash is running on http://127.0.0.1:8079/

